Henter ut artikler og ingressen fra VG-artikler i Norsk Aviskorpus

Artikler uten Ingress eller innhold (ofte bak betalingsmur) beholdes ikke

URL, dato og kategorier beholdes og

In [2]:
import tarfile
import glob
import os

import xml.etree.ElementTree as ET

import tqdm
import pandas as pd


In [3]:
for nak_file in glob.glob("data/*.tar"):
    print(nak_file)
    with tarfile.open(nak_file, "r") as f:
        f.extractall("data/extracted/")

data\nak_2012.tar
data\nak_2013.tar
data\nak_2014.tar
data\nak_2015.tar
data\nak_2016.tar
data\nak_2017.tar
data\nak_2018.tar
data\nak_2019.tar


In [5]:
for vg_nak in tqdm.tqdm(glob.glob("data/extracted/*/vg-*-nob.tar.gz")):
    with tarfile.open(vg_nak, "r") as f:
        f.extractall("vg_nob_nak")

100%|██████████| 8/8 [04:00<00:00, 30.04s/it]


In [6]:
vg_articles = glob.glob("vg_nob_nak/*/VG*.xml")
print(f"{len(vg_articles)} VG-artikler funnet")

195777 VG-artikler funnet


In [7]:
def extract_from_xml(xml_tree):
    article = ""
    title = ""
    url = ""
    date = ""
    classes = ""

    div_elms = xml_tree.findall("*//div")
    for elm in div_elms:
        div_type = elm.attrib.get("type")
        if div_type == "ingress":
            ingress = elm.text.strip() if elm.text else None
        elif div_type == "text":
            sentences = elm.findall("p")
            article += " ".join(sentence.text.strip() for sentence in sentences)
        elif div_type == "title" and elm.attrib.get("level") == "1":
            title = elm.text.strip()

    if not article or not ingress:
        return None

    attr_elms = xml_tree.findall("*//attribute")
    for elm in attr_elms:
        attr_name = elm.attrib.get("name")
        if attr_name == "url":
            url = elm.attrib.get("value")
        elif attr_name == "date":
            date = elm.attrib.get("value")
        elif attr_name == "class1":
            classes = elm.attrib.get("value")


    return {"title": title, "url": url, "published": date, "classes": classes, "article": article, "ingress": ingress}


In [14]:
vg_summarization_dataset = pd.DataFrame(columns=["title", "url", "published", "classes", "article", "ingress"])

for vg_article in tqdm.tqdm(vg_articles):
    with open(vg_article, "r", encoding="utf-8") as xml_file:
        try:
            xml_tree = ET.fromstring(xml_file.read())
        except Exception as e:
            print(vg_article)
            print(e)
        extracted = extract_from_xml(xml_tree)
        if extracted:
            vg_summarization_dataset.loc[len(vg_summarization_dataset)] = extracted


 12%|█▏        | 23012/195777 [01:56<24:15, 118.72it/s]

vg_nob_nak\vg-2012-nob\VG-20120926-10046247.xml
undefined entity &euro;: line 43, column 33


 18%|█▊        | 35350/195777 [05:20<49:25, 54.09it/s]  

vg_nob_nak\vg-2013-nob\VG-20130220-10100892.xml
undefined entity &euro;: line 44, column 44


 20%|██        | 40043/195777 [07:00<48:31, 53.50it/s]  

vg_nob_nak\vg-2013-nob\VG-20130416-10109427.xml
undefined entity &euro;: line 25, column 6


 35%|███▍      | 67845/195777 [19:16<1:08:40, 31.05it/s]

vg_nob_nak\vg-2014-nob\VG-20140819-23277706.xml
undefined entity &euro;: line 25, column 45


100%|██████████| 195777/195777 [1:48:13<00:00, 30.15it/s]  


In [16]:
vg_summarization_dataset.to_csv("vg_summarization.csv")

In [19]:
vg_summarization_dataset

,title,url,published,classes,article,ingress
0,Hit bør du reise i 2012,http://www.vg.no/reise/artikkel.php?artid=1002...,2012-01-01 15:01,reise,De aller fleste av oss vil i hvert fall bruke ...,366 flunkende nye dager fulle av muligheter. M...
1,Reddet livet til mirakeljenta,http://www.vg.no/nyheter/innenriks/22-juli/art...,2012-01-01 10:38,"nyheter,innenriks,22-juli",Sikkerhetsvaktens dramatiske og høyst unike bi...,(VG Nett) I minuttene etter at bomben gikk av ...
2,- Tomrommet etter Ege blir vanskelig å fylle,http://www.vg.no/sport/haandball/artikkel.php?...,2012-01-01 09:45,"sport,haandball",- EM blir det store høydepunktet denne sesonge...,(VG Nett) Drammen-spiller Lars Erik Bjørnsen s...
3,Slik takler kjendisene julefråtsingen,http://www.vg.no/rampelys/artikkel.php?artid=1...,2012-01-01 12:08,rampelys,Artist AYLAR LIE (27) har lagt seg i hardtreni...,Mens Kathrine Sørland gjerne kutter mat en dag...
4,Northug sov seg gjennom rakettsmellene,http://www.vg.no/sport/ski/langrenn/northug/ar...,2012-01-01 10:50,"sport,ski,langrenn,northug",På årets første dag står en av Northugs favori...,OBERSTDORF (VG NETT) Petter Northug (25) er kl...
...,...,...,...,...,...,...
187617,Michelle Williams (39) er gravid – og forlovet...,http://www.vg.no/rampelys/i/Jo2Gm4/michelle-wi...,2019-12-31 08:11,rampelys,"Nyheten om at Michelle Williams er gravid, bek...",«Dawson’s Creek»-skuespilleren har forlovet se...
187618,Nyttårstrafikken: Sterk vind og glatte veier f...,http://www.vg.no/nyheter/innenriks/i/6jvdE3/ny...,2019-12-31 08:20,"nyheter,innenriks",Tirsdag morgen melder flere politidistrikt om ...,2019 ser ut til å bli avsluttet med mye vind o...
187619,Raus med skrytet,http://www.vg.no/nyheter/meninger/i/pLEb7W/rau...,2019-12-31 06:31,"nyheter,meninger",På årets siste dag er det mange å ta farvel me...,Monica Mæland mener kommunereformen står til t...
187620,Skogbrannene i Australia: Setter inn militærhe...,http://www.vg.no/nyheter/utenriks/i/pLEmvW/sko...,2019-12-31 07:49,"nyheter,utenriks","I Mallacoota, sør i delstaten Victoria, har fo...",SYDNEY/OSLO (VG) Skogbrannene i Australia fort...


In [18]:
vg_summarization_dataset.memory_usage()

Index        1500976
title        1500976
url          1500976
published    1500976
classes      1500976
article      1500976
ingress      1500976
dtype: int64